In [ ]:
import geopandas as gpd
import glob
import os
from osgeo import ogr
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
points=gpd.read_file('Points.shp')
points.plot()

In [ ]:
points

In [ ]:
buffer_dist=115 #buffer distance = image size x spatial resolution /2

# Read in the shapefile
points = gpd.read_file("Points.shp")

# Create square buffers with a side length of buffer_dist units around the point features
points['geometry'] = points.buffer(buffer_dist)

points['geometry'] = points.geometry.envelope

# Save the new shapefile
points.to_file("squares.shp")

In [ ]:
# Split the flooded and nonflooded points 
#points_flooded=points[points['Label']==1]
#points_notflooded=points[points['Label']==0]

# Iterate over each feature in the shapefile
for index, feature in points.iterrows():
    # Create a new GeoDataFrame with just the current feature
    #print(index)
    
    feature_gdf = points.iloc[[index]]
    #print(feature_gdf)
    #print(feature_gdf['Label'][index])
    
    
        
    # Save the feature to a new shapefile
    feature_gdf.to_file(r"divided\feature_{}.shp".format(index))

In [ ]:
# Read raster files with GDAL
# import 
ds = gdal.Open("Composite_raster.tif") # open a raster with several bands, each band  represent one predictive feature
gt= ds.GetGeoTransform() #get the transformation data
proj = ds.GetProjection() #get the projection

band = ds.GetRasterBand(1) #read the first band 
array = band.ReadAsArray() #read the first band as an array

plt.figure()  #plot the raster to check that you every thing is working well
plt.imshow(array)

In [ ]:
# change the path to the folder where we saved the splitted polygons
shp_path=r"D:\divided"
os.chdir(shp_path)
shp_file = glob.glob('*.shp')
#index =0
for file in shp_file:
        #print(str(file))         
        ds2 = ogr.Open(file, 1)
        layer = ds2.GetLayer()
        shp_ds=gpd.read_file(file)
        #print(shp_ds['Label'][0])
        #index+=1
        # we will clip the raster with each polygon and save the flooded and notflooded locations in different folders
        # we will check the label, if label =0 then this is not flooded location
        if shp_ds['Label'][0] == 0 : # clip and save not flooded locations
             #Save the feature to a new shapefile
            dsClip = gdal.Warp(r"D:\Predictive_features\NotFlooded\feature_"+str(file[:-4])+".tif", ds, cutlineDSName = file,
                       cropToCutline = True, dstNodata = np.nan)
        else: # clip and save flooded locations
            # Save the feature to a new shapefile
                  
            dsClip = gdal.Warp(r"D:\Predictive_features\Flooded\feature_"+str(file[:-4])+".tif", ds, cutlineDSName = file,
                       cropToCutline = True, dstNodata = np.nan)